In [ ]:
# 1. Core PyTorch first
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118  

# 2. Utility libs
!pip install matplotlib timm

# 3. Segment Anything
!pip install git+https://github.com/facebookresearch/segment-anything.git


# 4. CLIP
!pip install ftfy regex

# 5. OpenCV (with contrib modules for tracker)
!pip install opencv-contrib-python



Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/facebookresearch/segment-anything.git to c:\users\hello\appdata\local\temp\pip-req-build-pzn1l2pd
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git 'C:\Users\Hello\AppData\Local\Temp\pip-req-build-pzn1l2pd'


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/CLIP.git to c:\users\hello\appdata\local\temp\pip-req-build-ina14zj0
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\Hello\AppData\Local\Temp\pip-req-build-ina14zj0'


Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import os
import numpy as np
import cv2
import torch
import timm
from segment_anything import sam_model_registry, SamPredictor
import timm
import torch.nn as nn
import torchvision.transforms as T
from PIL import Image


In [3]:
!curl.exe -L "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth" -o "sam_vit_b.pth"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  357M    0 1485k    0     0  1805k      0  0:03:22 --:--:--  0:03:22 1824k
  1  357M    1 4152k    0     0  2324k      0  0:02:37  0:00:01  0:02:36 2336k
  2  357M    2 8344k    0     0  3084k      0  0:01:58  0:00:02  0:01:56 3093k
  3  357M    3 11.4M    0     0  3171k      0  0:01:55  0:00:03  0:01:52 3179k
  3  357M    3 14.1M    0     0  3082k      0  0:01:58  0:00:04  0:01:54 3088k
  4  357M    4 17.3M    0     0  3111k      0  0:01:57  0:00:05  0:01:52 3331k
  5  357M    5 20.4M    0     0  3128k      0  0:01:57  0:00:06  0:01:51 3420k
  6  357M    6 23.6M    0     0  3152k      0  0:01:56  0:00:07  0:01:49 3188k
  7  357M    7 26.8M    0     0  3154k      0  0:01:56  0:00:08  0:01:48 3143k
  8  357M    8 29.6M    0     0  3131k      0  0:01

In [4]:
import os
print(os.path.exists("sam_vit_b.pth"))  # Should be True


True


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# load model (point to the checkpoint you downloaded)
sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b.pth")
sam.to(device)
predictor = SamPredictor(sam)

In [18]:
# --------------------------
# 2. CLIP (Option 3: re-implement + load weights)
# --------------------------
!pip install git+https://github.com/openai/CLIP.git
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

# import open_clip
# clip_model, _, clip_preprocess = open_clip.create_model_and_transforms(
#     'ViT-B-32', pretrained='openai'
# )
# clip_model = clip_model.eval().cuda()

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/CLIP.git to c:\users\hello\appdata\local\temp\pip-req-build-_fdp0qeb
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\Hello\AppData\Local\Temp\pip-req-build-_fdp0qeb'


In [ ]:
clip_model = clip_model.eval().to(device)

In [20]:
# --------------------------
# 3. DINO (Option 3: re-implement + load weights)
# --------------------------
import timm
dino_model = timm.create_model('vit_base_patch16_224', pretrained=True)
dino_model.eval().to(device)

# Preprocess for DINO
dino_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])


In [22]:
# --------------------------
# Helper functions
# --------------------------

def get_sam_mask(image, click_point):
    predictor.set_image(image)
    masks, _, _ = predictor.predict(
        point_coords=np.array([click_point]),
        point_labels=np.array([1]),
        multimask_output=False
    )
    return masks[0]

def get_clip_score(segment_crop, text_query):
    img_proc = clip_preprocess(segment_crop).unsqueeze(0).cuda()
    text = open_clip.tokenize([text_query]).cuda()
    with torch.no_grad():
        img_feat = clip_model.encode_image(img_proc)
        txt_feat = clip_model.encode_text(text)
        img_feat /= img_feat.norm(dim=-1, keepdim=True)
        txt_feat /= txt_feat.norm(dim=-1, keepdim=True)
        score = (img_feat @ txt_feat.T).item()
    return score

def get_dino_score(segment_crop, ref_image):
    img1 = dino_transform(segment_crop).unsqueeze(0).cuda()
    img2 = dino_transform(ref_image).unsqueeze(0).cuda()
    with torch.no_grad():
        f1 = dino_model(img1)
        f2 = dino_model(img2)
        f1 /= f1.norm(dim=-1, keepdim=True)
        f2 /= f2.norm(dim=-1, keepdim=True)
        score = (f1 @ f2.T).item()
    return score

In [ ]:
# --------------------------
# Interactive Part
# --------------------------

cap = cv2.VideoCapture("video.mp4")  # or 0 for webcam
ret, frame = cap.read()
cap.release()

image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
Image.fromarray(image_rgb).show()  # display first frame

# ---- Choose mode ----
mode = widgets.Dropdown(
    options=['Click', 'Text', 'Reference'],
    description='Mode:',
)
display(mode)

def run_segmentation(choice):
    if choice == 'Click':
        x = int(input("Enter X coordinate: "))
        y = int(input("Enter Y coordinate: "))
        mask = get_sam_mask(image_rgb, (x,y))
        crop = Image.fromarray(image_rgb[y:y+mask.shape[0], x:x+mask.shape[1]])
        display(crop)
        print("Segmented object from click point.")

    elif choice == 'Text':
        text_query = input("Enter text query: ")
        # Take full image mask first (dummy click in center)
        h,w,_ = image_rgb.shape
        mask = get_sam_mask(image_rgb, (w//2,h//2))
        x, y, w, h = cv2.boundingRect(mask.astype(np.uint8))
        crop = Image.fromarray(image_rgb[y:y+h, x:x+w])
        score = get_clip_score(crop, text_query)
        display(crop)
        print(f"CLIP score for '{text_query}': {score:.4f}")

    elif choice == 'Reference':
        uploader = widgets.FileUpload(accept='image/*', multiple=False)
        display(uploader)
        print("Upload a reference image, then rerun scoring cell.")

run_button = widgets.Button(description="Run Segmentation")
display(run_button)

def on_click(b):
    run_segmentation(mode.value)

run_button.on_click(on_click)